In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants as sc
import copy

from DeepFMKit import physics
from DeepFMKit.experiments import Experiment 

# This factory function must be defined at the top level of a cell
# to be pickle-able for multiprocessing.
def create_distortion_configs(params: dict) -> dict:
    """
    Takes a dictionary of trial parameters and returns the fully configured
    physics objects needed for the simulation.
    """
    m_main = params['m_main']
    m_witness = params['m_witness']
    distortion_amp = params['distortion_amp']
    distortion_phase = params.get('distortion_phase', 0.0) # Default to 0 if not stochastic

    # Create the base config objects
    laser_config = physics.LaserConfig()
    main_ifo_config = physics.InterferometerConfig()
    
    # Configure the objects based on the input parameters
    opd_main = main_ifo_config.meas_arml - main_ifo_config.ref_arml
    if opd_main == 0: opd_main = 0.2 # Avoid division by zero
    laser_config.df = (m_main * sc.c) / (2 * np.pi * opd_main)
    laser_config.df_2nd_harmonic_frac = distortion_amp
    laser_config.df_2nd_harmonic_phase = distortion_phase

    # Create the witness interferometer configuration
    witness_ifo_config = physics.InterferometerConfig()
    if laser_config.df > 0 and 'm_witness' in params:
        opd_witness = (m_witness * sc.c) / (2 * np.pi * laser_config.df)
        witness_ifo_config.ref_arml = 0.01
        witness_ifo_config.meas_arml = witness_ifo_config.ref_arml + opd_witness
        f0 = sc.c / laser_config.wavelength
        static_fringe_phase = (2 * np.pi * f0 * opd_witness) / sc.c
        witness_ifo_config.phi = (np.pi / 2.0) - static_fringe_phase
    
    return {
        'laser_config': laser_config,
        'main_ifo_config': main_ifo_config,
        'witness_ifo_config': witness_ifo_config
    }

In [ ]:
# --- 1. Declaratively Define the Experiment ---
exp = Experiment(description="W-DFMI Bias Correction vs. Modulation Non-Linearity")

# Define the single parameter sweep axis
exp.add_axis('distortion_amp', np.linspace(0.00, 0.1, 11))

# Define parameters that are fixed for the entire experiment
exp.set_static({
    'm_main': 20.0,
    'm_witness': 0.04
})

# Define the Monte Carlo simulation by specifying a stochastic variable
exp.n_trials = 50
exp.add_stochastic_variable('distortion_phase', lambda: np.random.uniform(0, 2 * np.pi))

# Tell the experiment which function to use to create the physics configs
exp.set_config_factory(create_distortion_configs)

# Define the analyses to run
exp.add_analysis(name='wdfmi_fit', fitter_method='wdfmi_ortho')
exp.add_analysis(name='nls_fit', fitter_method='nls', fitter_kwargs={'ndata': 30, 'parallel': False})

print("Experiment configured successfully.")

In [ ]:
# --- 2. Run the Experiment ---
# The .run() method handles all the parallelization and result aggregation.
if __name__ == "__main__":
    results = exp.run()

In [ ]:
# Calculate the bias for each analysis after the fact
m_true = exp.static_params['m_main']
nls_bias = results['nls_fit']['m']['mean'] - m_true
wdfmi_bias = results['wdfmi_fit']['m']['mean'] - m_true

nls_std = results['nls_fit']['m']['std']
wdfmi_std = results['wdfmi_fit']['m']['std']

fig, ax = plt.subplots(figsize=(12, 7))

ax.errorbar(
    results['axes']['distortion_amp'] * 100, 
    nls_bias, 
    yerr=nls_std,
    fmt='o-', capsize=5, color='tab:red', label='Standard NLS Fitter Bias'
)
ax.errorbar(
    results['axes']['distortion_amp'] * 100, 
    wdfmi_bias, 
    yerr=wdfmi_std,
    fmt='s-', capsize=5, color='tab:green', label='W-DFMI Fitter Bias (Corrected)'
)

ax.axhline(0, color='k', linestyle='--', linewidth=1, alpha=0.7)
ax.set_xlabel('2nd Harmonic Distortion Amplitude (%)', fontsize=14)
ax.set_ylabel(r"Bias in Modulation Depth, $\delta m = \hat{m} - m_{\rm true}$ (rad)", fontsize=14)
ax.set_title(exp.description, fontsize=16)
ax.grid(True, which='both', linestyle=':')
ax.legend(fontsize=12)
plt.tight_layout()
plt.show()